In [77]:
!pip install plac
!pip install spacy
!python -m spacy download en_core_web_md 

     |████████████████████████████████| 45.7 MB 433 kB/s  eta 0:00:01     |████████████████▉               | 24.0 MB 12.0 MB/s eta 0:00:02


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [104]:
from __future__ import unicode_literals, print_function
import pickle
import plac
import random
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding
from spacy.training import Example
import srsly
import pickle
import os
import csv
import numpy as np
from datetime import datetime

In [93]:
# creating a folder to hold the data files, model results, output etc.
model_dir_name = "custom_entity_extractor"
model_dir_parent = "/Users/ash/Desktop"
output_dir = os.path.join(model_dir_parent, model_dir_name)
os.makedirs(output_dir, exist_ok=True)
print("created", output_dir)

created /Users/ash/Desktop/custom_entity_extractor


In [94]:
# below code is to convert a jsonl file in this format to spacy format:
# {"order":10,"Used?":"staged","Dataset name":"checkin-three","ignore?":"","reason":"","relation count":"many","complexity":"complex","original_md_text":"We also know that emotional distress during pregnancy and mood disorders, which can be triggered as women face the uncertainty that climate change brings to their lives and the lives of their families, can affect not only the physical but also the neurological and cognitive development of the unborn child.\n","text":"We also know that emotional distress during pregnancy and mood disorders, which can be triggered as women face the uncertainty that climate change brings to their lives and the lives of their families, can affect not only the physical but also the neurological and cognitive development of the unborn child. IMPLIED_BASE IMPLIED_BASE","characters":308,"document_index":14,"md_sentence_index":35,"classification_origin":"cm-label-eval","url":"https://www.climaterealityproject.org/blog/dose-reality-how-climate-change-affects-our-kids-straight-pediatrician","_input_hash":690545709,"_task_hash":1928498526,"spans":[{"start":8,"end":12,"token_start":2,"token_end":2,"label":"confidence"},{"start":18,"end":27,"token_start":4,"token_end":4,"label":"type_of"},{"start":28,"end":36,"token_start":5,"token_end":5,"label":"base"},{"start":37,"end":53,"token_start":6,"token_end":7,"label":"when"},{"start":58,"end":62,"token_start":9,"token_end":9,"label":"type_of"},{"start":63,"end":72,"token_start":10,"token_end":10,"label":"base"},{"start":80,"end":83,"token_start":13,"token_end":13,"label":"confidence"},{"start":87,"end":96,"token_start":15,"token_end":15,"label":"change_direction"},{"start":97,"end":99,"token_start":16,"token_end":16,"label":"predicate"},{"start":100,"end":105,"token_start":17,"token_end":17,"label":"to_whom"},{"start":115,"end":126,"token_start":20,"token_end":20,"label":"base"},{"start":132,"end":146,"token_start":22,"token_end":23,"label":"base"},{"start":147,"end":153,"token_start":24,"token_end":24,"label":"predicate"},{"start":192,"end":200,"token_start":33,"token_end":33,"label":"to_whom"},{"start":202,"end":205,"token_start":35,"token_end":35,"label":"confidence"},{"start":206,"end":212,"token_start":36,"token_end":36,"label":"predicate"},{"start":226,"end":234,"token_start":40,"token_end":40,"label":"type_of"},{"start":248,"end":260,"token_start":44,"token_end":44,"label":"type_of"},{"start":265,"end":274,"token_start":46,"token_end":46,"label":"type_of"},{"start":275,"end":306,"token_start":47,"token_end":51,"label":"base"}],"tokens":[{"text":"We","start":0,"end":2,"id":0,"ws":true,"disabled":false},{"text":"also","start":3,"end":7,"id":1,"ws":true,"disabled":false},{"text":"know","start":8,"end":12,"id":2,"ws":true,"disabled":false},{"text":"that","start":13,"end":17,"id":3,"ws":true,"disabled":false},{"text":"emotional","start":18,"end":27,"id":4,"ws":true,"disabled":false},{"text":"distress","start":28,"end":36,"id":5,"ws":true,"disabled":false},{"text":"during","start":37,"end":43,"id":6,"ws":true,"disabled":false},{"text":"pregnancy","start":44,"end":53,"id":7,"ws":true,"disabled":false},{"text":"and","start":54,"end":57,"id":8,"ws":true,"disabled":false},{"text":"mood","start":58,"end":62,"id":9,"ws":true,"disabled":false},{"text":"disorders","start":63,"end":72,"id":10,"ws":false,"disabled":false},{"text":",","start":72,"end":73,"id":11,"ws":true,"disabled":false},{"text":"which","start":74,"end":79,"id":12,"ws":true,"disabled":false},{"text":"can","start":80,"end":83,"id":13,"ws":true,"disabled":false},{"text":"be","start":84,"end":86,"id":14,"ws":true,"disabled":false},{"text":"triggered","start":87,"end":96,"id":15,"ws":true,"disabled":false},{"text":"as","start":97,"end":99,"id":16,"ws":true,"disabled":false},{"text":"women","start":100,"end":105,"id":17,"ws":true,"disabled":false},{"text":"face","start":106,"end":110,"id":18,"ws":true,"disabled":false},{"text":"the","start":111,"end":114,"id":19,"ws":true,"disabled":false},{"text":"uncertainty","start":115,"end":126,"id":20,"ws":true,"disabled":false},{"text":"that","start":127,"end":131,"id":21,"ws":true,"disabled":false},{"text":"climate","start":132,"end":139,"id":22,"ws":true,"disabled":false},{"text":"change","start":140,"end":146,"id":23,"ws":true,"disabled":false},{"text":"brings","start":147,"end":153,"id":24,"ws":true,"disabled":false},{"text":"to","start":154,"end":156,"id":25,"ws":true,"disabled":false},{"text":"their","start":157,"end":162,"id":26,"ws":true,"disabled":false},{"text":"lives","start":163,"end":168,"id":27,"ws":true,"disabled":false},{"text":"and","start":169,"end":172,"id":28,"ws":true,"disabled":false},{"text":"the","start":173,"end":176,"id":29,"ws":true,"disabled":false},{"text":"lives","start":177,"end":182,"id":30,"ws":true,"disabled":false},{"text":"of","start":183,"end":185,"id":31,"ws":true,"disabled":false},{"text":"their","start":186,"end":191,"id":32,"ws":true,"disabled":false},{"text":"families","start":192,"end":200,"id":33,"ws":false,"disabled":false},{"text":",","start":200,"end":201,"id":34,"ws":true,"disabled":false},{"text":"can","start":202,"end":205,"id":35,"ws":true,"disabled":false},{"text":"affect","start":206,"end":212,"id":36,"ws":true,"disabled":false},{"text":"not","start":213,"end":216,"id":37,"ws":true,"disabled":false},{"text":"only","start":217,"end":221,"id":38,"ws":true,"disabled":false},{"text":"the","start":222,"end":225,"id":39,"ws":true,"disabled":false},{"text":"physical","start":226,"end":234,"id":40,"ws":true,"disabled":false},{"text":"but","start":235,"end":238,"id":41,"ws":true,"disabled":false},{"text":"also","start":239,"end":243,"id":42,"ws":true,"disabled":false},{"text":"the","start":244,"end":247,"id":43,"ws":true,"disabled":false},{"text":"neurological","start":248,"end":260,"id":44,"ws":true,"disabled":false},{"text":"and","start":261,"end":264,"id":45,"ws":true,"disabled":false},{"text":"cognitive","start":265,"end":274,"id":46,"ws":true,"disabled":false},{"text":"development","start":275,"end":286,"id":47,"ws":true,"disabled":false},{"text":"of","start":287,"end":289,"id":48,"ws":true,"disabled":false},{"text":"the","start":290,"end":293,"id":49,"ws":true,"disabled":false},{"text":"unborn","start":294,"end":300,"id":50,"ws":true,"disabled":false},{"text":"child","start":301,"end":306,"id":51,"ws":false,"disabled":false},{"text":".","start":306,"end":307,"id":52,"ws":true,"disabled":false},{"text":"IMPLIED_BASE","start":308,"end":320,"id":53,"ws":true,"disabled":false},{"text":"IMPLIED_BASE","start":321,"end":333,"id":54,"ws":false,"disabled":false}],"_session_id":"checkin_three_all_labels-answers","_view_id":"relations","relations":[{"head":4,"child":5,"head_span":{"start":18,"end":27,"token_start":4,"token_end":4,"label":"type_of"},"child_span":{"start":28,"end":36,"token_start":5,"token_end":5,"label":"base"},"color":"#c5bdf4","label":"Concept_Member"},{"head":9,"child":10,"head_span":{"start":58,"end":62,"token_start":9,"token_end":9,"label":"type_of"},"child_span":{"start":63,"end":72,"token_start":10,"token_end":10,"label":"base"},"color":"#c5bdf4","label":"Concept_Member"},{"head":46,"child":51,"head_span":{"start":265,"end":274,"token_start":46,"token_end":46,"label":"type_of"},"child_span":{"start":275,"end":306,"token_start":47,"token_end":51,"label":"base"},"color":"#c5bdf4","label":"Concept_Member"},{"head":44,"child":51,"head_span":{"start":248,"end":260,"token_start":44,"token_end":44,"label":"type_of"},"child_span":{"start":275,"end":306,"token_start":47,"token_end":51,"label":"base"},"color":"#c5bdf4","label":"Concept_Member"},{"head":40,"child":51,"head_span":{"start":226,"end":234,"token_start":40,"token_end":40,"label":"type_of"},"child_span":{"start":275,"end":306,"token_start":47,"token_end":51,"label":"base"},"color":"#c5bdf4","label":"Concept_Member"},{"head":7,"child":5,"head_span":{"start":37,"end":53,"token_start":6,"token_end":7,"label":"when"},"child_span":{"start":28,"end":36,"token_start":5,"token_end":5,"label":"base"},"color":"#c5bdf4","label":"Concept_Member"},{"head":7,"child":10,"head_span":{"start":37,"end":53,"token_start":6,"token_end":7,"label":"when"},"child_span":{"start":63,"end":72,"token_start":10,"token_end":10,"label":"base"},"color":"#c5bdf4","label":"Concept_Member"},{"head":15,"child":10,"head_span":{"start":87,"end":96,"token_start":15,"token_end":15,"label":"change_direction"},"child_span":{"start":63,"end":72,"token_start":10,"token_end":10,"label":"base"},"color":"#c5bdf4","label":"Concept_Member"},{"head":15,"child":5,"head_span":{"start":87,"end":96,"token_start":15,"token_end":15,"label":"change_direction"},"child_span":{"start":28,"end":36,"token_start":5,"token_end":5,"label":"base"},"color":"#c5bdf4","label":"Concept_Member"},{"head":17,"child":5,"head_span":{"start":100,"end":105,"token_start":17,"token_end":17,"label":"to_whom"},"child_span":{"start":28,"end":36,"token_start":5,"token_end":5,"label":"base"},"color":"#c5bdf4","label":"Concept_Member"},{"head":17,"child":10,"head_span":{"start":100,"end":105,"token_start":17,"token_end":17,"label":"to_whom"},"child_span":{"start":63,"end":72,"token_start":10,"token_end":10,"label":"base"},"color":"#c5bdf4","label":"Concept_Member"},{"head":17,"child":20,"head_span":{"start":100,"end":105,"token_start":17,"token_end":17,"label":"to_whom"},"child_span":{"start":115,"end":126,"token_start":20,"token_end":20,"label":"base"},"color":"#c5bdf4","label":"Concept_Member"},{"head":33,"child":20,"head_span":{"start":192,"end":200,"token_start":33,"token_end":33,"label":"to_whom"},"child_span":{"start":115,"end":126,"token_start":20,"token_end":20,"label":"base"},"color":"#c5bdf4","label":"Concept_Member"},{"head":23,"child":20,"head_span":{"start":132,"end":146,"token_start":22,"token_end":23,"label":"base"},"child_span":{"start":115,"end":126,"token_start":20,"token_end":20,"label":"base"},"color":"#ffd882","label":"Contributes_To"},{"head":20,"child":5,"head_span":{"start":115,"end":126,"token_start":20,"token_end":20,"label":"base"},"child_span":{"start":28,"end":36,"token_start":5,"token_end":5,"label":"base"},"color":"#ffd882","label":"Contributes_To"},{"head":20,"child":10,"head_span":{"start":115,"end":126,"token_start":20,"token_end":20,"label":"base"},"child_span":{"start":63,"end":72,"token_start":10,"token_end":10,"label":"base"},"color":"#ffd882","label":"Contributes_To"},{"head":5,"child":51,"head_span":{"start":28,"end":36,"token_start":5,"token_end":5,"label":"base"},"child_span":{"start":275,"end":306,"token_start":47,"token_end":51,"label":"base"},"color":"#ffd882","label":"Contributes_To"},{"head":10,"child":51,"head_span":{"start":63,"end":72,"token_start":10,"token_end":10,"label":"base"},"child_span":{"start":275,"end":306,"token_start":47,"token_end":51,"label":"base"},"color":"#ffd882","label":"Contributes_To"}],"answer":"accept"}
# above is only one entry from the jsonl file

In [95]:
# Create Training data
# 70% training data and 30% validation/testing data

# spacy requires data to be in a specific format which is what the code below does. Here is the acceptable formart:
# [('Heat stress and persistent dehydration can cause kidney damage. IMPLIED_BASE IMPLIED_BASE', {'entities': [(5, 11, 'base'), (27, 38, 'base'), (49, 55, 'base')]})]

file_name_answers = "main_3_per_cluster_download.66081fcf-3ef5-48ea-97e0-49298d29b477"
file_path_answers = "/Users/ash/Desktop/" + file_name_answers + ".jsonl"
data = srsly.read_jsonl(file_path_answers)
final_sent = []

for entry in data:
    if "text" in entry:
        text = entry["text"]
    else:
        text = ""
        throw("NO 'text' field encountered!")

    label_arr = []
    label_tup = ()
    if entry['answer'] == "accept":
        for relation in entry['spans']:
            if ("label" in relation) and ("start" in relation) and ("end" in relation):
                child_span_start = relation["start"]
                child_span_end = relation["end"]
                word = text[child_span_start:child_span_end]
                if relation["label"] == "base":
                    tmp_tuple = (child_span_start, child_span_end, relation["label"])
                    label_arr.append(tmp_tuple)
    if len(label_arr) > 0:
        label_tup = (text, {"entities": label_arr})
        final_sent.append(label_tup)

training_data_index = int(len(final_sent)*0.7)

with open(os.path.join(output_dir, 'training_data.txt'), 'wb') as file:
    pickle.dump(final_sent[:training_data_index], file)
    
with open(os.path.join(output_dir, 'testing_data.txt'), 'wb') as file:
    pickle.dump(final_sent[training_data_index:], file)

In [96]:
LABEL = ['base']

with open (os.path.join(output_dir, 'training_data.txt'), 'rb') as file:
    TRAIN_DATA = pickle.load(file)

with open (os.path.join(output_dir, 'testing_data.txt'), 'rb') as file:
    TEST_DATA = pickle.load(file)

In [102]:
model = "en_core_web_md"
new_model_name = "custom_entity_extraction"
n_iter = 1000
batch_size = 500

In [98]:
print(len(TRAIN_DATA))
print(len(TEST_DATA))

674
289


In [99]:
TRAIN_DATA

[('These risks are crop insurance, health care, wildfire suppression, hurricane-related disaster relief, and federal facility flood risk, all of which are anticipated to cost billions of dollars more by the end of the century due to the impacts of climate change.23 IMPLIED_BASE IMPLIED_BASE',
  {'entities': [(21, 30, 'base'),
    (32, 43, 'base'),
    (45, 65, 'base'),
    (85, 100, 'base'),
    (123, 128, 'base'),
    (245, 262, 'base')]}),
 ('The IPCC reported that Alexandria’s beaches would be submerged even with a 0.5-metre sea-level rise, while 8 million people would be displaced by flooding in Alexandria and the Nile Delta if no protective measures are taken. IMPLIED_BASE IMPLIED_BASE',
  {'entities': [(36, 43, 'base'),
    (85, 88, 'base'),
    (117, 123, 'base'),
    (146, 154, 'base')]}),
 ('(Later, a severe mid-summer Arctic heat wave contributed to historic melting of the Greenland ice sheet, with 12.5 billion tons of ice melting into the ocean on a single day — the “biggest

In [ ]:
print("starting time: " + datetime.now.strftime("%d/%m/%Y %H:%M:%S"))
if model is not None:
    nlp = spacy.load(model)
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('en')
    print("Created blank 'en' model")
    
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner)
else:
    ner = nlp.get_pipe('ner')

for i in LABEL:
    # Adding new entity
    ner.add_label(i)

if model is None:
    optimizer = nlp.begin_training()
else:
    #optimizer = nlp.entity.create_optimizer()
    optimizer = nlp.create_optimizer()
    
# Get names of other pipes to disable them during training to train only NER
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        batches = minibatch(TRAIN_DATA, size=compounding(4., 32., 1.001))
        #for batch in batches:
        #    texts, annotations = zip(*batch)
        #    nlp.update(texts, annotations, sgd=optimizer, drop=0.2, losses=losses)
        #print('Losses', losses)
        for batch in spacy.util.minibatch(TRAIN_DATA, size=batch_size):
            for text, annotations in batch:
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)
                nlp.update([example], drop=0.35, sgd=optimizer, losses=losses)
        print("Final loss: ", losses)
        
# Save model 
if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.meta['name'] = new_model_name  # rename model
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)

print("ending time: " + datetime.now.strftime("%d/%m/%Y %H:%M:%S"))

Loaded model 'en_core_web_md'
{'ner': 335.3068620017908}
{'ner': 600.1878094972557}
{'ner': 846.3339350705864}
{'ner': 1103.2596986401836}
{'ner': 1331.159771941905}
{'ner': 1572.8550863649166}
{'ner': 1758.39196340218}
{'ner': 1952.854849994267}
{'ner': 2158.465987622825}
{'ner': 2338.705719485127}
{'ner': 2524.0918335023703}
{'ner': 2721.5211542745164}
{'ner': 2915.0009362358555}


In [ ]:
# Test the saved model
print("Loading from", output_dir)
nlp_test = spacy.load(output_dir)

In [ ]:
result = os.path.join(output_dir, "result.csv")
all_bases = 0
correct_bases = 0
false_pos = 0
false_neg = 0

headers = ["text", "predicted_base", "actual_base"]
final_res = []
final_res.append(headers)
for sent in TEST_DATA:
    doc2 = nlp_test(sent[0])
    res = [i.text for i in list(doc2.ents)]
    actual_base = []
    for tokens in sent[1]['entities']:
        actual_base.append(sent[0][tokens[0]:tokens[1]])
    all_bases += len(actual_base)
    correct_bases += len(np.intersect1d(actual_base, res))
    false_pos += len(list(sorted(set(actual_base) - set(res))))
    false_neg += len(list(sorted(set(res) - set(actual_base))))
    final_res.append([sent, res, actual_base])

acc = ((correct_bases*100)/all_bases)
print("False pos: ", false_pos)
print("False neg: ", false_neg)
print("Marked {} out of {} bases correctly. Accuracy: {}".format(correct_bases, all_bases, acc))
    
with open(result, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(final_res)

print("created result file", result)